In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction, Box, Rgba, WeldJoint
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from scipy.spatial.transform import Rotation as sp_rot

from pydrake.all import RationalForwardKinematics, ModelInstanceIndex, SpatialInertia, RevoluteJoint, FixedOffsetFrame
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
meshcat = StartMeshcat()
meshcat.Delete()

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.js' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend.runfiles/drake/geometry/meshcat.js' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not s

In [8]:
tmp = sp_rot.random().as_euler("xyz")
random_rp = np.array([tmp[0],0, tmp[1]])
tmp = RotationMatrix(RollPitchYaw(random_rp))

In [34]:
meshcat.Delete()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)

link_dimensions = (0.1, 0.2, 0.001)
pendulum_box = Box(*link_dimensions)
pendulum_inertia = SpatialInertia.SolidBoxWithDensity(1, *link_dimensions)
pendulum_model_instance = plant.AddModelInstance("pendulum")
def AddPendulumLink(parent_frame, link_index: int, color = np.array([1,0,0,1])):
    link_name = f"link_{link_index}"
    pend_body = plant.AddRigidBody(link_name, pendulum_model_instance, pendulum_inertia)
    parent_to_child_attach_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,link_dimensions[1]/2, 0)))
    )
    next_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,-link_dimensions[1]/2, 0)))
    )
    # add the revolute joint,
    plant.AddJoint(RevoluteJoint(
        f"joint_{link_index}", parent_frame, parent_to_child_attach_frame, [0,0,1],
        -np.pi, np.pi,
    )
    )
    
    plant.RegisterCollisionGeometry(pend_body, RigidTransform(),
                                    pendulum_box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(pend_body, RigidTransform(),
                                pendulum_box, link_name, color)
    return next_frame

def N_Link_Pendulum(n: int):
    assert n > 1
    parent_frame = plant.world_frame()
    colors = np.array(viz_utils.n_colors(n))/255
    colors = np.hstack([colors,np.ones((colors.shape[0],1))])
    for i in range(n):
        parent_frame = AddPendulumLink(parent_frame, i+1, colors[i])

        
obstacle_model_instance = plant.AddModelInstance("obstacle")
def AddRandomBox(index: int, pos_limits, size_limits, color = [0,0,0,1]):
    link_name = f"obstacle_{index}"     
    
    center = np.append(np.random.uniform(pos_limits[0],pos_limits[1], 2),0)
    
    random_rp = np.zeros(3)
    lim = np.pi/10
    random_rp[2] = np.random.uniform(-lim, lim)
    angle = RotationMatrix(RollPitchYaw(random_rp))
    
    pose = RigidTransform(p=center)@RigidTransform(R=angle)
    
    dimensions = np.append(np.random.uniform(size_limits[0],size_limits[1],2),0.001)
    box = Box(*dimensions)
    inertia = SpatialInertia.SolidBoxWithDensity(1, *dimensions)
    
    body = plant.AddRigidBody(link_name, obstacle_model_instance, inertia)
    plant.AddJoint(WeldJoint(link_name, plant.world_frame(), body.body_frame(),
                            pose))
    plant.RegisterCollisionGeometry(body, RigidTransform(),
                                    box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(body, RigidTransform(),
                                box, link_name, color)

def AddNRandomBoxes(n: int, pos_limits=(-10,10), size_limits = (0.1,1), color= [0,0,0,1]):
    assert n > 1
    for i in range(n):
        AddRandomBox(i, pos_limits, size_limits, color)

# child_frame = AddPendulumLink(plant.world_frame(), 1)
num_pend = 10
N_Link_Pendulum(num_pend)

num_obstacles = 10
obstacle_pos_limits = link_dimensions[1]*num_pend*np.array([-1,1])
obstacle_size_limits = (0.01,0.5)
AddNRandomBoxes(num_obstacles, obstacle_pos_limits, obstacle_size_limits)

plant.Finalize()

meshcat.Set2dRenderMode(RigidTransform(p=np.array([0,0,1])))

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

## Set up the sliders so we can move the plant around manually

In [21]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0.0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q1', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q2', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q3', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q4', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q5', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q6', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q7', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q8', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='q9', max=3.141592653589793, min=-3.141592653589793)

In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)

height = 0.1
ground_box = Box(10,10,0.1)
ground_offset = RigidTransform(p=[0,0,-height/2])
friction = CoulombFriction(1,1)
ground_collision = plant.RegisterCollisionGeometry(
    plant.world_body(), ground_offset, ground_box, "ground_col", friction
)
ground_visual = plant.RegisterVisualGeometry(
    plant.world_body(), ground_offset, ground_box, "ground", [1,0,0,0]
)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.75
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,0.4], [x_fac,-1.4*sp,0.4], [-x_fac,-1.4*sp,0.4], [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True


diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good)
DEBUG:drak

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.ico' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/geometry/meshcat.ico' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel'

In [6]:
#compute limits in s-space
limits_s = []
lower_q = plant.GetPositionLowerLimits()
upper_q = plant.GetPositionUpperLimits()

lower_q[0] = -np.pi/2
upper_q[0] = np.pi/2

lower_q[-1] = 0
upper_q[-1] = 0

lower_q[2] = 0

for q in [lower_q,upper_q]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)
print(limits_s)

[[ -1.          -1.7320606    0.          -1.7320606  -11.43007018
   -1.7320606    0.        ]
 [  1.           1.7320606   11.43007018   1.7320606   11.43007018
    1.7320606    0.        ]]


In [7]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_col_context)
def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1
    
def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

In [8]:
prm_important_samples_q = np.array([
    [-0.47, -0.19, 0.34, -1.89, 0.68, 0.18, 0],
    [-2.13, -1.44,  1.53, -1.9 , -2.97,  0.6 ,  0.22],
    [2.22, -2.09, -1.61, -1.61, -2.87,  0.64,  3.05],
    [0.33,  0.41, -0.17, -1.79,  1.13, -0.49,  3.05]
]) 

prm_important_samples_s = [ Ratfk.ComputeSValue(x, q_star) for x in prm_important_samples_q]

In [9]:
print(len(prm_important_samples_s))

3


In [10]:
tries = 15
for q in prm_important_samples_q:
    s = Ratfk.ComputeSValue(q, q_star)
    for _ in range(tries):
        new_s = s + 1e-2*np.random.randn(plant.num_positions()) 
        if not check_collision_s_by_query(s):
            prm_important_samples_s.append(new_s)


In [11]:
len(prm_important_samples_s)

33

In [14]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]



def collision(pos, col_func_handle):
    return col_func_handle(pos)

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos)

prm_col_fn_handle = partial(collision, col_func_handle = check_collision_s_by_query)
# prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = check_collision_s_by_ik)

PRM = prm.PRM( 
            limits_s,
            num_points = 1000,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 3, 
            dist_thresh = 10,
            num_col_checks = 10,
            max_it = 1e3,
            initial_points = prm_important_samples_s.copy(),
            verbose = True,
            plotcallback = None,
    sample_via_gaussian = True,
    gaussian_sample_var = 1e-1
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
# PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
print(tot_num_edges)

body = plant.GetBodyByName("iiwa_link_7")
def visualize_task_space_trajectory(segment_start, segment_end,
                                    body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 100, path = "prm/seg"):
    s_waypoints = np.linspace(segment_start, segment_end, num_points)
    points = []
    for s in s_waypoints:
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(path, pc, point_size = path_size, rgba=color)


def plot_prm(PRM, body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(prefix)
    for idx, (s0, s1) in enumerate(PRM.prm_pairs):
        visualize_task_space_trajectory(s0, s1, body, color,
                                        path_size, num_points, 
                                        f"{prefix}/seg_{idx}")
def plot_prm_nodes(PRM, body, color = Rgba(0,0,1,1),
                                    point_size = 0.05, prefix = "prm_nodes"):
    meshcat.Delete(prefix)
    points = []
    for idx, s in enumerate(PRM.nodes):
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(prefix, pc, point_size = point_size, rgba=color)
    
# plot_prm(PRM, body)
plot_prm_nodes(PRM, body)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Samples 420
[PRM] Samples 450
[PRM] Samples 480
[PRM] Samples 510
[PRM] Samples 540
[PRM] Samples 570
[PRM] Samples 600
[PRM] Samples 630
[PRM] Samples 660
[PRM] Samples 690
[PRM] Samples 720
[PRM] Samples 750
[PRM] Samples 780
[PRM] Samples 810
[PRM] Samples 840
[PRM] Samples 870
[PRM] Samples 900
[PRM] Samples 930
[PRM] Samples 960
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected: 280
[PRM] Nodes 

In [14]:
def make_line_polys(PRM):
    polys = np.empty(shape=(plant.num_positions(), len(PRM.prm_pairs)), dtype = object)
    for i, (s0, s1) in enumerate(PRM.prm_pairs):
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))# PolynomialCommon(np.array([s1[j],s0[j]-s1[j]]))
            if i == 0 and j == 0:
                print(polys[j,i].EvaluateUnivariate(0))
                print(s0[j])
                print()
                print(polys[j,i].EvaluateUnivariate(1))
                print(s1[j])
                print()
    return polys

path_safe = make_line_polys(PRM)
# path_unsafe = make_line_polys(PRM_bad)
    

-0.22750850506154593
-0.22750850506154593

-0.2394237037094491
-0.2394237037094491



In [ ]:
# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 4)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")


cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [ ]:
t0 = time.time()
ret = cspace_free_path.FindSeparationCertificateGivenPath(path_safe, set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

In [ ]:
def plot_pair_safety(pairs, is_safe_vec, body,
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(f"{prefix}_safe")
    meshcat.Delete(f"{prefix}_unsafe")
    for idx, (s0, s1) in enumerate(pairs):
        if is_safe_vec[idx]:
            visualize_task_space_trajectory(s0, s1, body, Rgba(0,1,0,0.5),
                                            path_size, num_points, 
                                            f"{prefix}_safe/seg_{idx}")
        else:
            visualize_task_space_trajectory(s0, s1, body, Rgba(1,0,0,0.5),
                                            path_size, num_points, 
                                            f"{prefix}_unsafe/seg_{idx}")
plot_pair_safety(PRM.prm_pairs, ret[0], body)

In [ ]:
unsafe_inds = np.where(np.logical_not(ret))[1]

In [ ]:
unsafe_traj = path_safe[:,unsafe_inds]

In [ ]:
unsafe_traj[:,0]

In [ ]:
def animate_traj_s(traj, steps, total_time = 2):
    sleep_time = total_time/steps
    # loop
    idx = 0
    going_fwd = True
    time_points = np.linspace(0, traj.end_time(), steps)
    frame_count = 0
    meshcat.StartRecording()
    for frame_count, t in enumerate(time_points):
        t0 = time.time()
        s = traj.value(t)
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        diagram_context.SetTime(frame_count *sleep_time)
        diagram.ForcedPublish(diagram_context)
        
        if scene_graph.HasCollisions():
            print("COLLISION DETECTED")
        t1 = time.time()
        pause = sleep_time - (t1 - t0)
        if pause > 0:
            time.sleep(pause)
    meshcat.StopRecording()
    meshcat.PublishRecording()

In [ ]:
unsafe_traj.shape

In [ ]:
from pydrake.all import PiecewisePolynomial
traj = PiecewisePolynomial(unsafe_traj[:,:3].T, list(range(unsafe_traj[:,:3].shape[1])))
print(traj.rows())
print(traj.cols())

In [ ]:
animate_traj_s(traj, 1000, 1)

In [ ]:
unsafe_traj.shape